In [3]:
import ibis
import pandas as pd

In [10]:
ibis.options.sql.default_limit = None
ibis.options.interactive = True

hdfs_conn = ibis.hdfs_connect(host='bottou03.sjc.cloudera.com')

ibis_conn = ibis.impala.connect(host='bottou01.sjc.cloudera.com',
                                    port=21050,
                                    hdfs_client=hdfs_conn)

In [5]:
pageviews_tbl = ibis_conn.table('wiki_pageviews', database='u_juliet')

What is in a project name? What does this data look like?

In [11]:
project_names = pageviews_tbl.project_name.distinct()
project_names

0          iu.m.d
1       he.zero.q
2          br.m.q
3          co.m.d
4          zero.f
5          ru.m.b
6             ltg
7          lv.m.b
8              am
9          vi.m.d
10           ay.d
11           tk.d
12           cr.q
13           is.d
14      iu.zero.d
15           uk.s
16        dv.zero
17        en.zero
18      tl.zero.d
19           kl.d
20           ln.b
21           wa.b
22           da.q
23         sv.m.n
24           la.s
25           sk.s
26             tt
27           ko.q
28             es
29             kw
          ...    
2288           an
2289           pi
2290       fo.m.s
2291      ha.zero
2292       uz.m.b
2293    kn.zero.s
2294    es.zero.v
2295        sco.m
2296         ks.m
2297       pt.m.q
2298         cy.m
2299       az.m.b
2300       Amanhã
2301          21º
2302          pnb
2303        wg-en
2304         pt.b
2305           be
2306    vi.zero.q
2307       qu.m.q
2308         wo.q
2309         ar.d
2310       uk.m.b
2311    wa.zero.d
2312      

Maybe we can understand this by finding the projects with the most pages. Let's group by porject name and then count the size of the groups.

In [17]:
project_page_counts = pageviews_tbl.group_by(pageviews_tbl.project_name).size().sort_by(('count', False))
project_page_counts

                project_name     count
0                         en  57774129
1                       en.m  38093008
2                         fr  11234880
3                         de  10624957
4                       ja.m   9061891
5                         ru   8835232
6                         ja   8238480
7                         es   7545728
8                       de.m   6931540
9                       es.m   6561233
10                      fr.m   5902997
11                        it   5161429
12                      it.m   4737148
13                      ru.m   4541935
14                 commons.m   3981603
15                        pl   3877222
16                        zh   3618730
17                        pt   3519056
18                   en.zero   3400072
19                      pt.m   2710694
20                      zh.m   2385815
21                        nl   2043556
22                      pl.m   1846846
23                      nl.m   1696595
24                       

To find something interesting, it'll help to understand the language. 

In [8]:
[name for name in project_names if 'en' in name]

['en.zero',
 'en.m.q',
 'en.m.v',
 'en.n',
 'en.zero.v',
 'en.zero.b',
 'ten.m',
 'en.m.s',
 'en.zero.n',
 'en.m.n',
 'en.zero.d',
 'en.m.d',
 'en.zero.s',
 'en.m.b',
 'en.zero.voy',
 'en.voy',
 'en.zero.q',
 'en.s',
 'en.d',
 'en.m.voy',
 'ten',
 'en.m',
 'en.q',
 'en.v',
 'en',
 'en.b',
 'wg-en']

The part of the project name after the '.' specifies a special type of wiki, let's just look at the standard wiki pages we know and love.

In [20]:
pageviews_tbl[pageviews_tbl.project_name == 'en'].limit(10)

  project_name                     page_name  monthly_total  hourly_total day  \
0           en                             !              1             0  31   
1           en                            !!              1             0  31   
2           en                           !!!              3             0  31   
3           en                !!!Fuck_You!!!              2             0  31   
4           en  !!!Fuck_You!!!_and_Then_Some              1             0  31   
5           en                   !!!_(album)              1             0  31   
6           en                        !BANG!              1             0  31   
7           en                  !Kung_people              1             0  31   
8           en                    !Oka_Tokat              2             0  31   
9           en                             "              1             0  31   

  hour month  year  
0   03    12  2015  
1   03    12  2015  
2   03    12  2015  
3   03    12  2015  
4  

In [21]:
en_page_views = pageviews_tbl[pageviews_tbl.project_name == 'en'].projection(['page_name',
                                                                              'monthly_total',
                                                                             'hourly_total',
                                                                             'day',
                                                                             'hour',
                                                                             'month',
                                                                             'year'])

In [ ]:
en_page_views.limit(10)